<a href="https://colab.research.google.com/github/RafaelBi/machine_learning/blob/master/vgg_keras_plates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import pandas as pd 
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation, Conv2D, MaxPooling2D

import os
print(os.listdir("drive/My Drive/Plates"))


In [0]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  print(
      '\n\nThis error most likely means that this notebook is not '
      'configured to use a GPU.  Change this in Notebook Settings via the '
      'command palette (cmd/ctrl-shift-P) or the Edit menu.\n\n')
  raise SystemError('GPU device not found')

In [0]:
path1 = os.path.join("drive/My Drive/Plates/train/cleaned")
path2 = os.path.join("drive/My Drive/Plates/train/dirty")

plt.imshow(cv2.imread(os.path.join(path1,os.listdir(path1)[1]))) #show

In [0]:
with tf.device('/device:GPU:0'):
  X = []
  y = []
  def create_train_data(path,n):
      for p in os.listdir(path):
          category = n
          img_array = cv2.imread(os.path.join(path,p))
          new_img_array = cv2.resize(img_array, dsize=(224, 224))
          X.append(new_img_array)
          y.append(category)
    
  create_train_data(path1,1)
  create_train_data(path2,0)


In [0]:
X = np.array(X)
y = np.array(y)

In [0]:
y

In [0]:
X.shape[1:]

In [0]:
X = X/255.0

In [0]:
with tf.device('/device:GPU:0'):
  model = Sequential()

  model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu", input_shape = X.shape[1:]))
  model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
  model.add(MaxPooling2D(pool_size=(2,2),strides=(2,2)))

  model.add(Flatten())
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=4096,activation="relu"))
  model.add(Dense(units=1, activation="sigmoid"))

model.summary()

In [0]:
with tf.device('/device:GPU:0'):
  history=[]
  model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
  #model.fit(X, y, epochs=10, batch_size=8, validation_split=0.2)
  history=model.fit(X, y, epochs=10, batch_size=4)

In [0]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['loss'], label='loss')

plt.xlabel('Epoch')
plt.ylabel('Accuracy')
#plt.ylim([0.5, 1])
plt.legend(loc='lower right')

In [0]:
path1 = os.path.join("drive/My Drive/Plates/test")
with tf.device('/device:GPU:0'):
  X_test = []
  y_test = []
  def create_test_data(path):
      for p in os.listdir(path):
          y_test.append(p.split(".")[0])
          img_array = cv2.imread(os.path.join(path,p))
          new_img_array = cv2.resize(img_array, dsize=(224, 224))
          X_test.append(new_img_array)
        
  create_test_data(path1)
  X_test = np.array(X_test)
  X_test = X_test/255


In [0]:
with tf.device('/device:GPU:0'):
  predictions = model.predict(X_test)

In [0]:
with tf.device('/device:GPU:0'):
  predicted_val = [int(round(p[0])) for p in predictions]

In [0]:
submission_df = pd.DataFrame({'id':y_test, 'label':predicted_val})

In [0]:
submission_df.label = submission_df.label.astype('category')
submission_df.label

In [0]:
submission_df.label.values.categories=['cleaned','dirty']

In [0]:
submission_df.head()

In [0]:
submission_df.to_csv("Plates/submission_plates.csv", index=False)